In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix


csv_path = "iris.csv"

Load data and clean, try loading without shuffle

In [125]:
def load_csv(csv_path : str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        dataset = pd.read_csv(csv_path, delimiter = ',', engine = 'python', header = None)
        dataset = dataset.sample(frac = 1, random_state = 42)
        data_x, data_y = dataset.iloc[:, :4].values.tolist(), dataset.iloc[:, -1].values.tolist()
        sub_dataset = pd.DataFrame({'x': data_x, 'y': data_y})
        return sub_dataset

In [126]:
valami = load_csv(csv_path)
valami

,x,y
0,"[6.1, 2.8, 4.7, 1.2]",1
1,"[5.7, 3.8, 1.7, 0.3]",0
2,"[7.7, 2.6, 6.9, 2.3]",2
3,"[6.0, 2.9, 4.5, 1.5]",1
4,"[6.8, 2.8, -1111, 1.4]",1
...,...,...
145,"[6.1, 2.8, 4.0, 1.3]",1
146,"[4.9, 2.5, 4.5, 1.7]",2
147,"[5.8, 4.0, 1.2, 0.2]",0
148,"[5.8, 2.6, 4.0, 1.2]",1


In [ ]:
np.mean(x,axis=0),np.var(x,axis=0)

In [ ]:
np.nanmean(x,axis=0),np.nanvar(x,axis=0)

In [ ]:
x[np.isnan(x)] = 3.5
x.shape

In [ ]:
np.mean(x,axis=0),np.var(x,axis=0)

In [ ]:
(x > 13.0).sum(), (x < 0.0).sum()

In [ ]:
x[np.where(np.logical_or(x > 13.0,x < 0.0))]

In [ ]:
less_than = np.where(x < 0.0)
higher_than = np.where(x > 13.0)
less_than,higher_than

In [ ]:
y = np.delete(y,np.where(x < 0.0)[0],axis=0)
y = np.delete(y,np.where(x > 13.0)[0],axis=0)
x = np.delete(x,np.where(x < 0.0)[0],axis=0)
x = np.delete(x,np.where(x > 13.0)[0],axis=0)
x.shape,y.shape

Train test split
try optim

In [ ]:
def train_test_split(features:np.ndarray,
                     labels:np.ndarray,
                     test_split_ratio:float) -> None:
        
    test_size = int(len(features) * test_split_ratio)
    train_size = len(features) - test_size
    assert len(features) == test_size + train_size, "Size mismatch!"

    x_train,y_train = features[:train_size,:],labels[:train_size]
    x_test,y_test = features[train_size:train_size+test_size,:], labels[train_size:train_size + test_size]
    return (x_train,y_train,x_test,y_test)

Distance

In [ ]:
def euclidean(points:np.ndarray,element_of_x:np.ndarray) -> np.ndarray:
    return np.sqrt(np.sum((points - element_of_x) ** 2, axis = 1))

Predict func

In [ ]:
def predict(x_train:np.ndarray,y_train:np.ndarray,x_test:np.ndarray,k:int) -> np.ndarray:
    labels_pred = []
    for x_test_element in x_test:
        distances = euclidean(x_train,x_test_element)
        distances = np.array(sorted(zip(distances,y_train)))
        label_pred = mode(distances[:k,1],keepdims=False).mode
        labels_pred.append(label_pred)
    return np.array(labels_pred,dtype=np.int32)

Metrics

In [ ]:
def accuracy(y_test:np.ndarray,y_preds:np.ndarray) -> float:
    true_positive = (y_test == y_preds).sum()
    return true_positive / len(y_test) * 100

In [ ]:
def plot_confusion_matrix(y_test:np.ndarray,y_preds:np.ndarray):
        conf_matrix = confusion_matrix(y_test,y_preds)
        sns.heatmap(conf_matrix,annot=True) 